In [1]:
import pandas as pd
import mlflow

from plots import plot_confusion_matrix, plot_roc_curve

# MLFlow

Como hemos explorado en clase, MLFlow es una herramienta de código abierto diseñada para gestionar de manera eficiente el ciclo de vida completo de los modelos de Machine Learning. Este conjunto de funcionalidades abarca varios aspectos fundamentales:

- **Tracking**: Registra de forma sistemática los resultados y parámetros de los modelos durante su entrenamiento, lo que permite compararlos fácilmente y comprender mejor su rendimiento.
- **Projects**: Empaqueta el código de manera completamente reproducible, lo que facilita la colaboración entre equipos y garantiza la portabilidad del código en diferentes entornos.
- **Models**: Proporciona herramientas para gestionar el versionado de modelos, así como para desplegarlos como endpoints de servicio. Esta capacidad resulta especialmente valiosa, ya que MLFlow ofrece integraciones para su despliegue en la nube. Además, permite exportar modelos compatibles con Apache Spark.

## Instalación de MLFlow

Para realizar este hands-on, necesitás tener instalado MLFlow en tu máquina local. Para ello, debemos instalar la librería `mlflow` dentro de un entorno virtual.

Una vez instalado, se debe ejecuta:

```Bash
mlflow server --host 127.0.0.1 --port 8888 
```

## MLFlow Tracking

### Conceptos importantes sobre MLFlow Tracking

Guardar el tracking de nuestros modelos en el servidor de MLFlow es muy sencillo. Simplemente hay que hacer:

In [2]:
mlflow.set_tracking_uri('http://localhost:8888')

Con esto, ya tendremos establecida la conexión con nuestro servidor de MLFlow.

Opcionalmente, podemos crear un experimento donde incluir los parámetros, en caso de no tener uno creado previamente. Cada proyecto diferente debería tener su propio experimento. Para ello, utilizaremos el método `create_experiment`:

In [3]:
experiment_name = "experiment_name"

if not mlflow.get_experiment_by_name(experiment_name):
    mlflow.create_experiment(name=experiment_name)

A continuación, se detallan los tipos de datos que podemos registrar en MLFlow:

- **Parámetros del modelo**: Corresponden a los parámetros utilizados en el modelo. Se registran usando el método `log_param`.
- **Métricas**: Se refieren a las métricas de rendimiento, tales como RMSE, accuracy, AUC, entre otras. Se registran utilizando el método `log_metric`.
- **Artefactos**: Permiten incluir archivos adicionales, como los datos de entrenamiento, imágenes generadas durante el entrenamiento, entre otros. Se registran mediante el método `log_artifact` o `log_figure`.
- **Modelos**: Permiten guardar los modelos entrenados. Se registran con el método `log_model`. Además, MLFlow ofrece técnicas de registro automático (autologging) compatibles con bibliotecas como Scikit-Learn, TensorFlow, Gluon, XGBoost, LightGBM y Statsmodels. Es decir, se puede utilizar el método `autolog`, y MLFlow registrará automáticamente los datos generados durante el proceso de entrenamiento.

Para poder registrar información, primero debemos indicar a MLFlow que inicie un proceso de seguimiento (run). Esto se puede hacer de dos maneras:

1. Usar el método `start_run` en conjunto con `with` para evitar cerrar el proceso manualmente:
```Python
with mlflow.start_run():
   mlflow.log_param('max_depth', max_depth)
```

2. Usar explícitamente los métodos `start_run()` y `end_run()`:
```Python
mlflow.start_run() 
mlflow.log_param('max_depth', max_depth) 
mlflow.end_run()
```

### Usando MLFlow con un ejemplo: Iris Dataset

Vamos a emplear MLFlow para entrenar dos modelos utilizando el conjunto de datos Iris. Los modelos que utilizaremos serán Logistic Regression y Random Forest, ambos de la librería Scikit-Learn. En esta notebook, nos enfocaremos únicamente en la parte de regresión logística.

In [4]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, accuracy_score, recall_score
from sklearn.model_selection import train_test_split

In [5]:
mlflow.set_tracking_uri('http://localhost:8888')

In [6]:
experiment_name = "experiment_iris"

if not mlflow.get_experiment_by_name(experiment_name):
    mlflow.create_experiment(name=experiment_name, 
                             tags={"project":"iris-dataset", 
                                   "team": "mlops1-fiuba"}) 

experiment = mlflow.get_experiment_by_name(experiment_name)

In [7]:
# Cargamos los datos
data = load_iris()

# Separamos entre evaluación y testeo
X_train, X_test, y_train, y_test = train_test_split(data['data'][:, :2], data['target'], test_size=0.2, random_state=42)

In [8]:
# Armamos los modelos base
model = LogisticRegression()

# Entrenamos el modelo
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


Registramos el modelo en MLFlow:

In [9]:
model_name = 'iris_lr'
run_name = "logistic_regression_exp"

mlflow.start_run(experiment_id = experiment.experiment_id, 
                 run_name=run_name,
                 tags={"model":"logistic_reg"})

# Logueamos los parametros del modelo
mlflow.log_params(model.get_params())

# Se obtiene las predicciones del dataset de evaluación
y_pred = model.predict(X_test)

# Se calculan las métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')

# Y las enviamos a MLflow
metrics ={
        'accuracy': accuracy,
        'precision': precision, 
        'recall': recall 
        }
mlflow.log_metrics(metrics)

# Como artefactos, obtenemos las gráficas de la curva ROC y la matriz de confusion
matrix_plot = plot_confusion_matrix(y_test, y_pred, save_path=None)
roc_plots = plot_roc_curve(y_test, model.predict_proba(X_test), save_path=None)

mlflow.log_figure(matrix_plot, artifact_file="matrix_plot.png")
mlflow.log_figure(roc_plots[0], artifact_file="roc_curve_1_plot.png")
mlflow.log_figure(roc_plots[1], artifact_file="roc_curve_2_plot.png")
mlflow.log_figure(roc_plots[2], artifact_file="roc_curve_3_plot.png")

# Registramos el modelo y los datos de entrenamiento
mlflow.sklearn.log_model(model, 
                         model_name,
                         input_example=X_test[0:1, :])

mlflow.end_run()

Accuracy: 0.9
Precision: 0.9013888888888889
Recall: 0.9


2025/07/12 20:54:28 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


🏃 View run logistic_regression_exp at: http://localhost:8888/#/experiments/302891218595240518/runs/e88e5f008fdb4f0387c793d3cf8e15e8
🧪 View experiment at: http://localhost:8888/#/experiments/302891218595240518


#### Realizar predicciones con un modelo de MLFlow

Para obtener predicciones, debemos ir al apartado de *Artifacts*, donde MLFlow indica cómo realizar inferencias con el modelo, ya sea utilizando Spark o Python. Copiamos ese fragmento de código y lo ejecutamos, proporcionando los datos que deseamos predecir.

In [10]:
runs_df = mlflow.search_runs(
            experiment_ids=experiment.experiment_id,
            filter_string=f"tags.mlflow.runName = '{run_name}'"   
        )

run_id = runs_df["run_id"].iloc[0]
model_uri = f"runs:/{runs_df['run_id'].iloc[0]}/{model_name}" 

# Cargamos el modelo usando el modulo de scikit-learn
loaded_model = mlflow.sklearn.load_model(model_uri)

# Predecimos usando un dataframe de Pandas
loaded_model.predict(pd.DataFrame(X_test))

array([1, 0, 2, 1, 2, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 2, 2, 1, 1, 2, 0, 1,
       0, 2, 2, 2, 2, 2, 0, 0])